In [98]:
""""
conda create neo4j 
pip install neo4j 

"""


'"\nconda create neo4j \npip install neo4j \n\n'

In [123]:
#Neo4j 서버 연결 확인 
from neo4j import GraphDatabase

# Neo4j 서버에 연결
uri = "bolt://localhost:7687"  # Neo4j 서버 URI (변수 이름 통일)
user = "neo4j"  # 사용자명
password = "a9862121!"  # 비밀번호

# URI와 사용자 정보 설정
AUTH = (user, password)


# GraphDatabase에 드라이버 생성 및 연결 확인
with GraphDatabase.driver(uri, auth=AUTH) as driver:  # 'uri'로 통일
    driver.verify_connectivity()  # 연결 확인
    print("Neo4j 서버와의 연결이 성공적으로 확인되었습니다.")

Neo4j 서버와의 연결이 성공적으로 확인되었습니다.


In [124]:
from neo4j import GraphDatabase


def flatten_dict(d, parent_key='', sep='_'):
    """
    딕셔너리를 재귀적으로 평탄화하는 함수
    """
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key + sep, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

# Neo4j에 연결하는 클래스 정의
class LegalGraphDB:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()
    
    def create_node(self, node_type, properties):
        with self.driver.session() as session:
            # 동적으로 Cypher 쿼리 생성
            query = f"CREATE (n:{node_type} {{"
            query += ", ".join([f"{key}: ${key}" for key in properties.keys()])
            query += "})"

            # 전달된 properties 딕셔너리를 사용하여 쿼리 실행
            session.run(query, **properties)
    
    # 노드 생성 메서드
    def create_law_node(self, node):
        with self.driver.session() as session:
            session.run(
                """
                CREATE (n:Article {
                    index: $index,
                    subtitle: $subtitle,
                    content: $content,
                    document_title: $document_title,
                    date: $date,
                    revise_info: $revise_info,
                    source: $source,
                    doc: $doc,
                    chapter: $chapter,
                    section: $section,
                    subsection: $subsection
                })
                """,
                index=node['index'],
                subtitle=node['subtitle'],
                content=node['content'],
                document_title=node['metadata']['document_title'],
                date=node['metadata']['date'],
                revise_info=node['metadata']['revise_info'],
                source=node['metadata']['source'],
                doc=node['metadata']['title']['doc'],
                chapter=node['metadata']['title']['chapter'],
                section=node['metadata']['title']['section'],
                subsection=node['metadata']['title']['subsection']
            )

    # REFERS_TO 관계 생성 메서드
    def create_relationship(self, from_index, to_index):
        with self.driver.session() as session:
            session.run(
                """
                MATCH (a:Article {index: $from_index}), (b:Article {index: $to_index})
                CREATE (a)-[:REFERS_TO]->(b)
                """,
                from_index=from_index,
                to_index=to_index
            )

    # 특정 키워드가 콘텐츠에 포함된 경우 REFERS_TO 관계 설정
    def create_references(self, nodes):
        for node in nodes:
            for target_node in nodes:
                if node['index'] != target_node['index'] and target_node['index'] in node['content']:
                    self.create_relationship(node['index'], target_node['index'])
    
    # 쿼리 실행 메서드
    """ 
    사용법 
    query = "MATCH (a:Article {index: $index}) RETURN a.index AS index, a.content AS content"
    db.run_query(query, parameters=None) 
    
    
    """
    def run_query(self, query, parameters=None):
        with self.driver.session() as session:
            result = session.run(query, parameters)
            return result
        

    
db = LegalGraphDB(uri, user, password)       



In [125]:
import json
import os


data_path = "../../results/merged_DCM_data.json"

#json 저장된 데이터 가져오기 
with open(data_path, "r", encoding="utf-8") as f:
    data = json.load(f)
    print(f"Loaded {len(data)} data from {data_path}")

Loaded 11 data from ../../results/merged_DCM_data.json


In [126]:

#노드 생성 
for (i, item) in enumerate(data):
    for (j, node) in enumerate(item) : 
        node = flatten_dict(item[j])
        
        if "시행령" in node["metadata_document_title"]:
            db.create_node("enforcement", node)
        elif "법률" in node["metadata_document_title"]:
            db.create_node("law", node)
        elif "시행규칙" in node["metadata_document_title"]:
            db.create_node("order", node)
        
print(f"{data_path}의 데이터를 Neo4j에 nodes생성 완료 ")


../../results/merged_DCM_data.json의 데이터를 Neo4j에 nodes생성 완료 


In [127]:
#edge 생성 
for d in data:
    db.create_references(d)
    
print(f"{data_path}의 데이터를 Neo4j에 edges 생성 완료 ")

../../results/merged_DCM_data.json의 데이터를 Neo4j에 edges 생성 완료 


In [133]:
# Get the index and content of all Articles with a specific index
# MATCH (n:law) RETURN n LIMIT 25 
records, summary, keys = driver.execute_query(
    "MATCH (n:law) RETURN n LIMIT 5",
    database_="neo4j",
)

# Loop through results and print them
for record in records:
    print(f"record: {record}")

# Summary information
print("The query `{query}` returned {records_count} records in {time} ms.".format(
    query=summary.query, records_count=len(records),
    time=summary.result_available_after,

))


record: <Record n=<Node element_id='4:b8573284-ccc5-44e4-9315-d15b0bd04d46:2039' labels=frozenset({'law'}) properties={'metadata_source': '국가법령정보센터', 'metadata_document_title': '자본시장과 금융투자업에 관한 법률 ( 약칭: 자본시장법 )\n', 'metadata_revise_info': '법률 제20305호, 2024. 2. 13., 일부개정\n', 'subtitle': '시행일', 'index': '제1조', 'metadata_date': '시행 2024. 8. 14.', 'content': '이 법은 공포 후 6개월이 경과한 날부터 시행한다.', 'metadata_title_supplementary': '부칙 <제20305호,2024. 2. 13.>'}>>
record: <Record n=<Node element_id='4:b8573284-ccc5-44e4-9315-d15b0bd04d46:2040' labels=frozenset({'law'}) properties={'metadata_source': '국가법령정보센터', 'metadata_document_title': '자본시장과 금융투자업에 관한 법률 ( 약칭: 자본시장법 )\n', 'metadata_revise_info': '법률 제20305호, 2024. 2. 13., 일부개정\n', 'subtitle': '유사투자자문업 신고의 직권말소에 관한 적용례', 'index': '제2조', 'metadata_date': '시행 2024. 8. 14.', 'content': '제101조제9항제1호의2의 개정규정은 이 법 시행 이후 공정거래위원회가 명한 시정조치를 이행하지 아니한 경우부터 적용한다.', 'metadata_title_supplementary': '부칙 <제20305호,2024. 2. 13.>'}>>
record: <Record n=<Node element_id=

C:\Users\Shic\AppData\Local\Temp\ipykernel_15832\4028000145.py:3: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query(
